### Import Required Libraries and Set Up Environment Variables

In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [ ]:
# Set environment variables from the .env in the local environment
###Day 2 Activity 2###

#load_dotenv() does not work for me, so I have to manually set the environment variables

NYT_API_KEY="EVdZqo5d74afd0baQEdvF7I4d1ASD9Rc"
TMDB_API_KEY="2103f54b8c28fe44793106711dd04b68"

### Access the New York Times API

In [ ]:
#######################################################################Day 2 Activity 6 for the next 2 cells####

# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

###################################################################requires you to read the NYT documentation####
# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
##############################################search for "Filter Query Examples" in the Article Search API docs.
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
###############################################look for "Limit Fields in Response in the Article Search API Docs"
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url= f'{url}api-key={NYT_API_KEY}&q={filter_query}&begin_date={begin_date}&end_date={end_date}'


In [ ]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0,20):
    query_url= f'{url}api-key={NYT_API_KEY}&q={filter_query}&begin_date={begin_date}&end_date={end_date}'
    # create query with a page number
    # API results show 10 articles at a time
    query_url = f'{query_url}&page={page}'
    
    # Make a "GET" request and retrieve the JSON
    reviews= requests.get(query_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f"Page {page} retrieved")

        # Print the page number that had no results then break from the loop
    except:
        print(f"no reviews on page {page}")


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
##########################This isn't an this weeks activity, but we have looked at the first five elements of a list before###
preview = reviews_list[:5]
reviews_json = json.dumps(reviews, indent=4)
reviews_json


In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
############################################################Day 1 Activity 8####
rl_df = pd.json_normalize(reviews_list)

In [ ]:
# Extract the title from the "headline.main" column and save it to a new column "title"
rl_df['title'] = rl_df['headline.main'].apply(lambda x: x['main'].split('\u2018', 1)[-1].split('\u2019 Review', 1)[0] if 'main' in x else None)



In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
##################you just applying the provided function to the keywords column

rl_df['keywords'] = rl_df['keywords'].apply(extract_keywords)

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list= rl_df['title'].to_list()

### Access The Movie Database API

In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
TMDB_API_KEY="2103f54b8c28fe44793106711dd04b68"

tmdb_key_string = "&api_key=" + TMDB_API_KEY

In [ ]:

# Create a request counter to sleep the requests after a multiple of 50 requests
for page in range(0,50):
    query_m_url = f"{url}{page}{tmdb_key_string}"
time.sleep(12)
# Create an empty list to store the results
movies=[]
# Loop through the titles
for m_title in title_list:
    query_m_url = f"{url}{m_title}{tmdb_key_string}"
    # Check if we need to sleep before making a request
    if sleep:
        time.sleep(12)
        
    # Perform a "GET" request for The Movie Database
    # Add 1 to the request counter
    response = requests.get(query_m_url).json()
    req_cnt += 1
    
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = response["results"][0]["id"]

        # Make a request for a the full movie details
        movie_url = (f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={TMDB_API_KEY}")

        # Execute "GET" request with url
        movie_response = requests.get(movie_url).json()
        
        # Extract the genre names into a list
        genres = []

        # Extract the spoken_languages' English name into a list
        spoken_languages = []

        # Extract the production_countries' name into a list
        production_countries = []

        # Add the relevant data to a dictionary and
        # append it to the movies list
        movies.append({
            "title": movie_response["title"],
            "release_date": movie_response["release_date"],
            "budget": movie_response["budget"],
            "revenue": movie_response["revenue"],
            "genres": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries
        })
        
        # Print out the title that was found
        print(f"Movie found: {movie_response['title']}")
        
    except:
        print(f"Movie not found for title: {m_title}")


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
m_title = movies[:5]
m_title_json = json.dumps(m_title, indent=4)
reviews_json

In [ ]:
# Convert the results to a DataFrame
m_df = pd.json_normalize(m_title)

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(rl_df, m_df, on='title', how='inner')

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['column1_faulty', 'column2_faulty', 'column3_faulty']  #
for column in columns_to_fix:
    merged_df[column] = merged_df[column].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
# Create a list of characters to remove
remove_c= ['[',']', '"']

# Loop through the list of columns to fix
for columns in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[columns] = merged_df[columns].astype(str)

    # Loop through characters to remove
    for character in remove_c:
            merged_df[columns] = merged_df[columns].str.replace(character, '')

# Display the fixed DataFrame
print(merged_df.head())

In [ ]:
# Drop "byline.person" column
merged_df = merged_df.dropna(columns=['byline.person'])

In [ ]:
# Delete duplicate rows and reset index
merged_df= merged-df.drop_duplicates().reset_index(drop=True)

In [ ]:
# Export data to CSV without the index
merged_df.to_csv("output/merged_df.csv", index=False)